<a href="https://colab.research.google.com/github/mmaguero/diploma_fpuna_nlp_ia/blob/master/2025/guarani-translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Transformers installation
! pip install transformers datasets evaluate accelerate
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00


# Translation

In [2]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/1JvfrvZgi6c?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/usr/local/lib/python3.12/dist-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


Translation converts a sequence of text from one language to another. It is one of several tasks you can formulate as a sequence-to-sequence problem, a powerful framework for returning some output from an input, like translation or summarization. Translation systems are commonly used for translation between different language texts, but it can also be used for speech or some combination in between like text-to-speech or speech-to-text.

This guide will show you how to:

1. Finetune [T5](https://huggingface.co/google-t5/t5-small) on the English-French subset of the [OPUS Books](https://huggingface.co/datasets/opus_books) dataset to translate English text to French.
2. Use your finetuned model for inference.

<Tip>

To see all architectures and checkpoints compatible with this task, we recommend checking the [task-page](https://huggingface.co/tasks/translation).

</Tip>

Before you begin, make sure you have all the necessary libraries installed:

```bash
pip install transformers datasets evaluate sacrebleu
```

We encourage you to login to your Hugging Face account so you can upload and share your model with the community. When prompted, enter your token to login:

In [24]:
from huggingface_hub import notebook_login

notebook_login()

## Load OPUS Books dataset

Start by loading the English-French subset of the [OPUS Books](https://huggingface.co/datasets/opus_books) dataset from the 🤗 Datasets library:

We will use the [jojajovai](https://github.com/pln-fing-udelar/jojajovai) corpus...

In [4]:
!git clone https://github.com/pln-fing-udelar/jojajovai

Cloning into 'jojajovai'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 16 (delta 1), reused 5 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (16/16), 3.52 MiB | 7.47 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [5]:
import pandas as pd

df = pd.read_csv("jojajovai/data/jojajovai_all.csv", encoding="utf-8")
train_dataset = df[df["split"]=="train"].to_dict("records")
test_dataset = df[df["split"]=="test"].to_dict()
dev_dataset = df[df["split"]=="dev"].to_dict()

train_dataset[0]
#test_dataset[0], dev_dataset[0]

{'split': 'train',
 'source': 'abc',
 'gn': 'Itaugua omokyre\'ÿ "omopotî" Congreso',
 'es': 'En Itauguá promueven "limpiar" el Congreso',
 'tokens_gn': "Itaugua omokyre'ÿ `` omopotî '' Congreso",
 'tokens_es': "En Itauguá promueven `` limpiar '' el Congreso"}

In [6]:
from datasets import Dataset, DatasetDict

# Create Dataset objects from the parsed data using Dataset.from_pandas
train_hf_dataset = Dataset.from_pandas(df[df["split"]=="train"])
dev_hf_dataset = Dataset.from_pandas(df[df["split"]=="dev"])
test_hf_dataset = Dataset.from_pandas(df[df["split"]=="test"])

# Create a DatasetDict
jojajovai = DatasetDict({
    "train": train_hf_dataset,
    "validation": dev_hf_dataset,
    "test": test_hf_dataset
})

jojajovai

DatasetDict({
    train: Dataset({
        features: ['split', 'source', 'gn', 'es', 'tokens_gn', 'tokens_es', '__index_level_0__'],
        num_rows: 20207
    })
    validation: Dataset({
        features: ['split', 'source', 'gn', 'es', 'tokens_gn', 'tokens_es', '__index_level_0__'],
        num_rows: 5314
    })
    test: Dataset({
        features: ['split', 'source', 'gn', 'es', 'tokens_gn', 'tokens_es', '__index_level_0__'],
        num_rows: 5334
    })
})

In [7]:
# push the dataset
#jojajovai.push_to_hub("mmaguero/jojajovai")

Split the dataset into a train and test set with the [train_test_split](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.train_test_split) method:

In [8]:
#books = books["train"].train_test_split(test_size=0.2)

Then take a look at an example:

In [9]:
#books["train"][0]

`translation`: an English and French translation of the text.

## Preprocess

In [10]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/XAR8jnZZuUs?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/usr/local/lib/python3.12/dist-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


The next step is to load a T5 tokenizer to process the English-French language pairs:

In [11]:
from transformers import AutoTokenizer

checkpoint = "Helsinki-NLP/opus-mt-tc-bible-big-mul-mul" # Switched to a smaller MarianMT model
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


The preprocessing function you want to create needs to:

1. Prefix the input with a prompt so T5 knows this is a translation task. Some models capable of multiple NLP tasks require prompting for specific tasks.
2. Set the target language (French) in the `text_target` parameter to ensure the tokenizer processes the target text correctly. If you don't set `text_target`, the tokenizer processes the target text as English.
3. Truncate sequences to be no longer than the maximum length set by the `max_length` parameter.

In [12]:
source_lang = "es"
target_lang = "gn"

# MarianMT models typically use ISO 639-1 codes directly
source_lang_nllb = "spa"
target_lang_nllb = "grn"

# MarianMT models usually don't require a specific prefix for translation
# Removing the prefix for this model?
prefix = f">>{target_lang_nllb}<< " # "Translate Spanish to Paraguayan Guarani: "

def preprocess_function(examples):
    # Set source and target language for the tokenizer
    # For MarianMT, src_lang and tgt_lang are often not explicitly set in this way
    # The model handles the language pair based on its training
    # We will still pass them for clarity, though it might not be strictly necessary for this tokenizer
    tokenizer.src_lang = source_lang_nllb
    tokenizer.tgt_lang = target_lang_nllb

    inputs = [prefix + example for example in examples[source_lang]]
    targets = [example for example in examples[target_lang]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=512, truncation=True)
    return model_inputs

To apply the preprocessing function over the entire dataset, use 🤗 Datasets [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) method. You can speed up the `map` function by setting `batched=True` to process multiple elements of the dataset at once:

In [13]:
tokenized_books = jojajovai.map(preprocess_function, batched=True)

Map:   0%|          | 0/20207 [00:00<?, ? examples/s]

Map:   0%|          | 0/5314 [00:00<?, ? examples/s]

Map:   0%|          | 0/5334 [00:00<?, ? examples/s]

Now create a batch of examples using [DataCollatorForSeq2Seq](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorForSeq2Seq). It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [14]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

## Evaluate

Including a metric during training is often helpful for evaluating your model's performance. You can quickly load a evaluation method with the 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) library. For this task, load the [SacreBLEU](https://huggingface.co/spaces/evaluate-metric/sacrebleu) metric (see the 🤗 Evaluate [quick tour](https://huggingface.co/docs/evaluate/a_quick_tour) to learn more about how to load and compute a metric):

In [15]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.4 MB/s eta 0:00:00


In [16]:
import evaluate

metric_bleu = evaluate.load("sacrebleu")
metric_chrf = evaluate.load("chrf")

Then create a function that passes your predictions and labels to [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) to calculate the SacreBLEU score:

In [17]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]

    # bleu
    result = metric_bleu.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    # chrf
    result["chrf"] = metric_chrf.compute(predictions=decoded_preds, references=decoded_labels)["score"]

    result["combined"] = result["bleu"] * 0.5 + result["chrf"] * 0.5
    result["gen_len"] = np.mean(prediction_lens)

    result = {k: round(v, 4) for k, v in result.items()}
    return result

Your `compute_metrics` function is ready to go now, and you'll return to it when you setup your training.

## Train

<Tip>

If you aren't familiar with finetuning a model with the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer), take a look at the basic tutorial [here](https://huggingface.co/docs/transformers/main/en/tasks/../training#train-with-pytorch-trainer)!

</Tip>

You're ready to start training your model now! Load T5 with [AutoModelForSeq2SeqLM](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForSeq2SeqLM):

In [18]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/991M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

At this point, only three steps remain:

1. Define your training hyperparameters in [Seq2SeqTrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Seq2SeqTrainingArguments). The only required parameter is `output_dir` which specifies where to save your model. You'll push this model to the Hub by setting `push_to_hub=True` (you need to be signed in to Hugging Face to upload your model). At the end of each epoch, the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) will evaluate the SacreBLEU metric and save the training checkpoint.
2. Pass the training arguments to [Seq2SeqTrainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Seq2SeqTrainer) along with the model, dataset, tokenizer, data collator, and `compute_metrics` function.
3. Call [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train) to finetune your model.

In [19]:
import gc
import torch

def cleanup():
    """Try to free GPU memory"""
    gc.collect()
    torch.cuda.empty_cache()

In [20]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
cleanup()

In [21]:
training_args = Seq2SeqTrainingArguments(
    output_dir="jojajovai-opus-mt-tc-bible-big-spa-grn",
    eval_strategy="epoch",
    num_train_epochs=3, # 2 a 10 o más
    #eval_strategy="steps",
    #max_steps=1000, # ideal=10K
    #save_steps=500,
    #logging_steps=500,
    learning_rate=2e-5,
    per_device_train_batch_size=8, # Further reduced batch size
    gradient_accumulation_steps=8, # Accumulate gradients over 4 steps
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    predict_with_generate=True,
    metric_for_best_model="combined",
    fp16=True, #change to bf16=True for XPU
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["validation"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mmaguero to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Bleu,Bleu Gen Len,Chrf,Chrf Gen Len,Combined
1,No log,2.018938,24.489600,37.184800,48.085200,37.184800,36.287400
2,1.974900,1.893737,25.536000,36.196500,49.654800,36.196500,37.595400
3,1.974900,1.861369,26.017900,36.500000,50.290700,36.500000,38.154300


TrainOutput(global_step=948, training_loss=1.8024167089019647, metrics={'train_runtime': 3514.479, 'train_samples_per_second': 17.249, 'train_steps_per_second': 0.27, 'total_flos': 5240810731732992.0, 'train_loss': 1.8024167089019647, 'epoch': 3.0})

In [22]:
trainer.evaluate(tokenized_books["test"])

{'eval_loss': 1.8078486919403076,
 'eval_bleu': 26.7899,
 'eval_bleu_gen_len': 36.4961,
 'eval_chrf': 51.656,
 'eval_chrf_gen_len': 36.4961,
 'eval_combined': 39.2229,
 'eval_runtime': 784.0177,
 'eval_samples_per_second': 6.803,
 'eval_steps_per_second': 0.851,
 'epoch': 3.0}

Once training is completed, share your model to the Hub with the [push_to_hub()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.push_to_hub) method so everyone can use your model:

In [25]:
trainer.push_to_hub()

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...spa-grn/model.safetensors:   0%|          |  215kB /  991MB            

  ...le-big-spa-grn/source.spm:  96%|#########6| 1.26MB / 1.32MB            

  ...le-big-spa-grn/target.spm:  96%|#########6| 1.26MB / 1.32MB            

  ...761036.24694a8f23c7.221.0:   8%|7         |   582B / 7.75kB            

  ...765334.24694a8f23c7.221.1:   7%|7         |  46.0B /   619B            

  ...spa-grn/training_args.bin:   7%|7         |   452B / 6.03kB            

CommitInfo(commit_url='https://huggingface.co/mmaguero/jojajovai-opus-mt-tc-bible-big-spa-grn/commit/21a0e58bccdc0c5bbf8ea8428c614f5f758e15ce', commit_message='End of training', commit_description='', oid='21a0e58bccdc0c5bbf8ea8428c614f5f758e15ce', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mmaguero/jojajovai-opus-mt-tc-bible-big-spa-grn', endpoint='https://huggingface.co', repo_type='model', repo_id='mmaguero/jojajovai-opus-mt-tc-bible-big-spa-grn'), pr_revision=None, pr_num=None)

<Tip>

For a more in-depth example of how to finetune a model for translation, take a look at the corresponding
[PyTorch notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/translation.ipynb).

</Tip>

## Inference

Great, now that you've finetuned a model, you can use it for inference!

Come up with some text you'd like to translate to another language. For T5, you need to prefix your input depending on the task you're working on. For translation from English to French, you should prefix your input as shown below:

In [26]:
text = ">>grn<< Las legumbres comparten recursos con bacterias fijadoras de nitrógeno."

The simplest way to try out your finetuned model for inference is to use it in a [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline). Instantiate a `pipeline` for translation with your model, and pass your text to it:

In [27]:
from transformers import pipeline

In [30]:
translator = pipeline("translation_spa_to_grn", model="mmaguero/jojajovai-opus-mt-tc-bible-big-spa-grn")
translator(text)

Device set to use cuda:0


[{'translation_text': "Ko'ã legumbre ombyatýva recurso umi bacteria fijadora nitrógeno."}]

In [34]:
translator(">>grn<< Asunción is one of the oldest cities in South America and the longest continually inhabited area in the Río de la Plata Basin.")

[{'translation_text': "Asunción ha'e peteî tavaguasu tujavéva América del Sur ha peteî tendáre opytáva Río de la Plata Basin-pe."}]

In [35]:
translator(">>spa<< Asunción is one of the oldest cities in South America and the longest continually inhabited area in the Río de la Plata Basin.")

[{'translation_text': 'Asunción es una de las ciudades más antiguas en América del Sur y la zona permanentemente habitada en el Río de la Plata Basin.'}]

In [41]:
eng=">>spa<< Asunción is one of the oldest cities in South America and the longest continually inhabited area in the Río de la Plata Basin."
to_spa=translator(eng)
to_grn=translator(f">>grn<< {to_spa[0]['translation_text']}")
eng, to_spa[0]['translation_text'], to_grn[0]['translation_text']

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


('>>spa<< Asunción is one of the oldest cities in South America and the longest continually inhabited area in the Río de la Plata Basin.',
 'Asunción es una de las ciudades más antiguas en América del Sur y la zona permanentemente habitada en el Río de la Plata Basin.',
 "Asunción ha'e tavaguasu ymaite guivéma América del Sur ha tenda permanentemente oikóva Río de la Plata Basin.")

In [48]:
to_grn_again = translator(f">>grn<< {to_grn[0]['translation_text']}")
to_grn_again

[{'translation_text': "Asunción ha'e tavaguasu ymaite guivéma América del Sur orekóva tenda permanentemente oikóva Río de la Plata Basin."}]

You can also manually replicate the results of the `pipeline` if you'd like:

Tokenize the text and return the `input_ids` as PyTorch tensors:

In [31]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mmaguero/jojajovai-opus-mt-tc-bible-big-spa-grn")
inputs = tokenizer(text, return_tensors="pt").input_ids

Use the [generate()](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationMixin.generate) method to create the translation. For more details about the different text generation strategies and parameters for controlling generation, check out the [Text Generation](https://huggingface.co/docs/transformers/main/en/tasks/../main_classes/text_generation) API.

In [32]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("mmaguero/jojajovai-opus-mt-tc-bible-big-spa-grn")
outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)

Decode the generated token ids back into text:

In [33]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'Ko’ã legumbre ombyatýva recurso umi bacteria fijadora nitrógeno.'